# 🎯 GitHub मोडेल (.NET) सँग योजना र डिजाइन ढाँचाहरू

## 📋 सिक्ने उद्देश्यहरू

यो नोटबुकले Microsoft Agent Framework प्रयोग गरेर .NET मा बुद्धिमान एजेन्टहरू निर्माण गर्नका लागि उद्यम-स्तरको योजना र डिजाइन ढाँचाहरू प्रदर्शन गर्दछ। तपाईंले जटिल समस्याहरूलाई टुक्रा पार्न, बहु-चरण समाधान योजना बनाउन, र .NET को उद्यम सुविधाहरू प्रयोग गरेर परिष्कृत कार्यप्रवाहहरू कार्यान्वयन गर्न सिक्नुहुनेछ।

## ⚙️ पूर्वआवश्यकता र सेटअप

**विकास वातावरण:**
- .NET 9.0 SDK वा उच्च संस्करण
- Visual Studio 2022 वा VS Code C# एक्सटेन्सनसहित
- GitHub Models API पहुँच

**आवश्यक निर्भरताहरू:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**पर्यावरण कन्फिगरेसन (.env फाइल):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
// #r "nuget: OpenAI, 2.4.0"

In [3]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0/Microsoft.Agents.AI.OpenAI.dll"

In [4]:
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [5]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [6]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [7]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;


In [8]:
 using DotNetEnv;

In [9]:
Env.Load("../../../.env");

In [10]:
System.Environment.GetEnvironmentVariable("GITHUB_ENDPOINT");

In [11]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [12]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [13]:


var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [14]:
const string AGENT_NAME ="TravelPlanAgent";

const string AGENT_INSTRUCTIONS = @"You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general request";

In [15]:

    public class Plan
    {
        [JsonPropertyName("assigned_agent")]
        public string? Assigned_agent { get; set; }

        [JsonPropertyName("task_details")]
        public string? Task_details { get; set; }
    }

In [16]:

    public class TravelPlan
    {
        [JsonPropertyName("main_task")]
        public string? Main_task { get; set; }

        [JsonPropertyName("subtasks")]
        public IList<Plan> Subtasks { get; set; }

    }

In [17]:

ChatClientAgentOptions agentOptions = new(name: AGENT_NAME, instructions: AGENT_INSTRUCTIONS)
{
    ChatOptions = new()
    {
        ResponseFormat = ChatResponseFormatJson.ForJsonSchema(
            schema: AIJsonUtilities.CreateJsonSchema(typeof(TravelPlan)),
            schemaName: "TravelPlan",
            schemaDescription: "Travel Plan with main_task and subtasks")
    }
};

In [18]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    agentOptions);

In [19]:

Console.WriteLine(await agent.RunAsync("Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne"));

{"main_task":"Plan a family trip from Singapore to Melbourne","subtasks":[{"assigned_agent":"FlightBooking","task_details":"Find and book flights from Singapore to Melbourne for a family of 4."},{"assigned_agent":"HotelBooking","task_details":"Find and book a family-friendly hotel in Melbourne."},{"assigned_agent":"CarRental","task_details":"Arrange car rental in Melbourne for local transportation."},{"assigned_agent":"ActivitiesBooking","task_details":"Book family-friendly activities and attractions in Melbourne."},{"assigned_agent":"DestinationInfo","task_details":"Provide information about Melbourne, including dining and family-friendly areas."}]}



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
